In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE203196"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE203196"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE203196.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE203196.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE203196.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from allergy and non allergy humans"
!Series_summary	"In previous studies with peripheral blood cells, platelet factors were found to be associated with severe allergic phenotypes. A reliable method yielding highly concentrated and pure platelet samples is usually not available for immunological studies. Plateletpheresis is widely used in the clinics for donation purposes."
!Series_summary	"In this study, we designed a protocol based on plateletpheresis to obtain platelet-rich plasma (PRP), Platelet Poor Plasma (PPP) as well as CD3+ and CD14+ cells matched samples from a waste plateletpheresis product for immunological studies."
!Series_overall_design	"Twenty-seven subjects were voluntarily subjected to plateletpheresis. Platelet-rich plasma (PRP) and blood cell concentrate contained in a leukocyte reduction system chamber (LRSC) were obtained in this process. CD3+ and CD14+ cells were isolated from the LRSC by density-gradient cen

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From background information, we see they performed "transcriptomic studies by Affymetrix"
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Allergies) is available in row 4
trait_row = 4

# Age is available in row 3
age_row = 3

# Gender is available in row 1
gender_row = 1

# 2.2 Data Type Conversion
# For trait: Converting "allergy: mild", "allergy: severe", "allergy: control" to binary (0/1)
def convert_trait(val):
    if pd.isna(val) or val is None:
        return None
    
    value = val.split(":", 1)[1].strip() if ":" in val else val.strip()
    
    # Convert to binary: 1 for any allergy, 0 for control
    if value == "control":
        return 0
    elif value in ["mild", "severe"]:
        return 1
    else:
        return None

# For age: Converting age to continuous value
def convert_age(val):
    if pd.isna(val) or val is None:
        return None
    
    value = val.split(":", 1)[1].strip() if ":" in val else val.strip()
    
    try:
        return float(value)
    except:
        return None

# For gender: Converting "F" to 0 and "M" to 1
def convert_gender(val):
    if pd.isna(val) or val is None:
        return None
    
    value = val.split(":", 1)[1].strip() if ":" in val else val.strip()
    
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability based on trait_row
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait data is available
if trait_row is not None:
    # We need to reconstruct the clinical data
    # Using the Sample Characteristics Dictionary from previous step
    # This assumes the preprocessed clinical data is available as a global variable
    # We'll continue with the code once we have access to the proper data
    
    # Print information about trait availability for debugging
    print(f"Trait row: {trait_row}, Trait is available: {is_trait_available}")
    print("To proceed with clinical feature extraction, we need the appropriate clinical data DataFrame.")
    print("Waiting for the correct data to be provided in the next step.")


Trait row: 4, Trait is available: True
To proceed with clinical feature extraction, we need the appropriate clinical data DataFrame.
Waiting for the correct data to be provided in the next step.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['16657436', '16657440', '16657445', '16657447', '16657450', '16657469',
       '16657473', '16657476', '16657480', '16657485', '16657489', '16657492',
       '16657502', '16657506', '16657509', '16657514', '16657527', '16657529',
       '16657534', '16657554'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be numeric probe IDs (16657436, 16657440, etc.) rather than 
# human gene symbols. Human gene symbols typically follow nomenclature like BRCA1, TP53, etc.
# These appear to be microarray probe IDs that would need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the gene identifiers in the gene expression data and the annotation data
# The gene expression data index contains values like '16657436', '16657440' which match
# the 'ID' column in the gene annotation dataframe

# The gene symbol information is in the 'gene_assignment' column, which contains gene symbols
# like DDX11L1, OR4F5, etc.

# 1. Identify the columns for probe ID and gene symbol
probe_id_column = 'ID'  # This is the column with identifiers matching our gene expression data
gene_symbol_column = 'gene_assignment'  # This column contains gene symbols

# 2. Extract gene mapping dataframe using get_gene_mapping function
# This will create a dataframe with 'ID' and 'Gene' columns
mapping_df = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)

# 3. Convert probe-level measurements to gene expression data
# The apply_gene_mapping function handles the many-to-many relationship
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the mapped gene data
print(f"Shape of gene expression data after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Shape of gene expression data after mapping: (81076, 30)
First 10 gene symbols after mapping:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), and Primary Myelofibrosis (PMF)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (23274, 30)


Normalized gene data saved to ../../output/preprocess/Allergies/gene_data/GSE203196.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM6161618': [1.0, 28.0, 0.0], 'GSM6161619': [1.0, 28.0, 0.0], 'GSM6161620': [1.0, 28.0, 0.0], 'GSM6161621': [1.0, 28.0, 0.0], 'GSM6161622': [1.0, 28.0, 0.0], 'GSM6161623': [1.0, 28.0, 0.0], 'GSM6161624': [1.0, 40.0, 0.0], 'GSM6161625': [1.0, 40.0, 0.0], 'GSM6161626': [1.0, 40.0, 0.0], 'GSM6161627': [0.0, 24.0, 1.0], 'GSM6161628': [0.0, 24.0, 1.0], 'GSM6161629': [0.0, 24.0, 1.0], 'GSM6161630': [1.0, 21.0, 0.0], 'GSM6161631': [1.0, 21.0, 0.0], 'GSM6161632': [1.0, 21.0, 0.0], 'GSM6161633': [1.0, 27.0, 0.0], 'GSM6161634': [1.0, 27.0, 0.0], 'GSM6161635': [1.0, 27.0, 0.0], 'GSM6161636': [1.0, 22.0, 0.0], 'GSM6161637': [1.0, 22.0, 0.0], 'GSM6161638': [1.0, 22.0, 0.0], 'GSM6161639': [1.0, 50.0, 1.0], 'GSM6161640': [1.0, 50.0, 1.0], 'GSM6161641': [1.0, 50.0, 1.0], 'GSM6161642': [0.0, 41.0, 0.0], 'GSM6161643': [0.0

Linked data shape after handling missing values: (30, 23277)
Checking for bias in trait distribution...
For the feature 'Allergies', the least common label is '0.0' with 9 occurrences. This represents 30.00% of the dataset.
The distribution of the feature 'Allergies' in this dataset is fine.

Quartiles for 'Age':
  25%: 24.0
  50% (Median): 27.5
  75%: 40.0
Min: 21.0
Max: 50.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 6 occurrences. This represents 20.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Allergies/GSE203196.csv
